# Tutaj raczej w ramach ciekawostki predykcje długości skoku wybranego zawodnika

Skok jest przewidywany za pomocą regresji liniowej na podstawie prędkości na progu, rekompensaty za wiatr,
punktu konstrukcyjnego skoczni oraz rozmiaru skoczni.
Problemem może być jednak to, że do momentu wyjścia z progu nie znamy prędkości zawodnika. Możemy ją jedynie przewidzieć na podstawie długości najazdu albo oszacować na podstawie uzyskiwanych prędkości na danej skoczni w zależności od długości najazdu, ale nie mam takich informacji.

Należy pamiętać, że główny wpływ na długośc skoku ma forma zawodnika.
Model przewiduje odległość, jaką zawodnik powinien osiągnąć będąc w swojej optymalnej dyspozycji, czyli w której
oddał najwięcej skoków w badanym okresie. Zatem model najlepiej będzie działał dla regularnych skoczków.

Dlatego świetne wyniki uzyskujemy dla Dawida Kubackiego, który jest jednym z najbardziej przewidywalnych skoczków.
Na przeciwnym biegunie jest Thomas Diethart, który był "skoczkiem jednego sezonu". Również dla Andreasa Wellingera w okresie 2020-2022 model nie przewidzi dobrze odległości, ponieważ rok 2021 to bardzo duży spadek formy tego zawodnika i skakanie na poziomie FIS Cup.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [2]:
comps = pd.read_csv("data/all_comps.csv")
names = pd.read_csv("data/all_names.csv")
results = pd.read_csv("data/all_results.csv")

In [3]:
merged = results.merge(comps, on="id")

In [4]:
def predict_jumper(name, down=2009, up=2022):
    """
    funkcja przewidująca długość skoku danego zawodnika
    name: nazwa skoczka w konwencji: nazwisko imię
    down: rok, od którego zaczynamy zbierać dane. Domyślnie 2009
    up: rok, do którego kończymy zbieranie danych. Domyślnie 2022
    """
    codex = names[names['name'] == name].iloc[0,1]
    dataset = merged[merged['codex_x'] == codex]
    dataset = dataset[['speed', 'wind_comp', 'k-point', 'hill_size_x', 'dist', 'season']].dropna()
    data = dataset[(dataset['season'] >= down) & (dataset['season'] <= up)]
    data = data[(data['speed'] != 0) & (data['dist'] != 0)]
    print(data[['speed', 'wind_comp', 'k-point', 'hill_size_x', 'dist']].corr())
    
    X = data.iloc[:, :4].values
    y = data.iloc[:, 4].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    y_pred = regressor.predict(X_test)
    
    print(y_pred)
    print(y_test)
    
    accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
    print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [5]:
predict_jumper('stoch kamil')

                speed  wind_comp   k-point  hill_size_x      dist
speed        1.000000   0.184218  0.910097     0.906525  0.890536
wind_comp    0.184218   1.000000 -0.008828    -0.009800 -0.066776
k-point      0.910097  -0.008828  1.000000     0.995369  0.956774
hill_size_x  0.906525  -0.009800  0.995369     1.000000  0.958227
dist         0.890536  -0.066776  0.956774     0.958227  1.000000
[132.22901804 126.41014402 130.47628104 132.56260493 128.4932799
 119.55614547 129.28640642 134.47618951 214.37058079 123.99154578
 208.4855157  124.26429987 135.36121246 124.8324399   89.39301993
 130.43263905 130.09479151 130.62032824 125.13136741 128.57167512
 140.31237244 132.88797968 119.49781555 130.29642241 118.52981478
 122.07751378 130.5429648  124.62587904 130.0474938  120.43956004
 123.91641329 129.68591463 125.372333   203.11057167 137.55116745
 128.69876482 121.77901787 218.48184197 124.14631169 224.95077371
 129.87919281 224.43576067 130.92516194 119.7761608  120.04907016
 123.465655